In [ ]:
import cv2
import os
import sys
from mrcnn import utils
from mrcnn import model as modellib
from mrcnn.config import Config
import mrcnn.model as modellib
from mrcnn.model import MaskRCNN
import uuid
import argparse
import skimage
import colorsys
import tensorflow as tf
import numpy as np
import shutil
import random
import argparse
import csv

class TestConfig(Config):
    NAME = "Deepfashion2"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 13
config = TestConfig()
 
path_dir = '/content/drive/MyDrive/real test/' + '스타일 카테고리'
file_list = os.listdir(path_dir)
output = '/content/drive/MyDrive/real test/' + '스타일 카테고리'

model = modellib.MaskRCNN(mode="inference", config=config, model_dir='/content/drive/MyDrive/Colab Notebooks/deepfashion/Deepfashion2_Training/')
model.load_weights('/content/drive/MyDrive/Colab Notebooks/deepfashion/Deepfashion2_Training/mask_rcnn_deepfashion2_075.h5', by_name=True)

class_names = ['short_sleeved_shirt', 'long_sleeved_shirt', 'short_sleeved_outwear', 'long_sleeved_outwear', 'vest', 'sling', 
               'shorts', 'trousers', 'skirt', 'short_sleeved_dress', 'long_sleeved_dress',
               'vest_dress', 'sling_dress']

def random_colors(N):
    np.random.seed(1)
    colors = [tuple(255 * np.random.rand(3)) for _ in range(N)]
    return colors

colors = random_colors(len(class_names))
class_dict = {
    name: color for name, color in zip(class_names, colors)
}

def apply_mask(image, mask, color, alpha=0.5):
    """apply mask to image"""
    for n, c in enumerate(color):
        image[:, :, n] = np.where(
            mask == 1,
            image[:, :, n] * (1 - alpha) + alpha * c,
            image[:, :, n]
        )
    return image

def display_instances(image, boxes, masks, ids, names, scores):
    n_instances = boxes.shape[0]
    print("no of potholes in frame :",n_instances)
    if not n_instances:
        print('NO INSTANCES TO DISPLAY')
    else:
        assert boxes.shape[0] == masks.shape[-1] == ids.shape[0]

    for i in range(n_instances):
        if not np.any(boxes[i]):
            continue
        y1, x1, y2, x2 = boxes[i]
        label = names[ids[i]-1]
        color = class_dict[label]
        score = scores[i] if scores is not None else None
        caption = '{} {:.2f}'.format(label, score) if score else label
        random_name = str(uuid.uuid4())
        mask = masks[:, :, i]  
        image = apply_mask(image, mask, color)
        image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        image = cv2.putText(image, caption, (x1, y1), cv2.FONT_HERSHEY_COMPLEX, 0.7, color, 2)
        
    return image

# display instances
for num in range(num_images):
    print(f'{num+1} / {num_images}')
    try:
      image_name = file_list[num]
      frame = cv2.imread(path_dir+'/'+image_name, cv2.IMREAD_COLOR)
      results = model.detect([frame], verbose=0)      
      r = results[0]
      display_instances(frame, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])
      random_name = str(uuid.uuid4())
      instance_image_name = random_name + ".jpg"
      cv2.imwrite(output + "/" + instance_image_name, frame)
    except:
      except_file.append(file_list[num])

print(f'except file: {except_file}')

def mask_crop(image, image_for_mask, masks, ids, names):
  n_instances = masks.shape[-1]
  print("number of items in frame :",n_instances)
   for i in range(masks.shape[-1]):
      print(masks.shape[-1])
      print(ids[i])
      mask = masks[:, :, i]
      image_for_mask[mask] = 255
      image_for_mask[~mask] = 0
      cropped = cv2.bitwise_and(image, image_for_mask)
      label = names[ids[i]]
      random_name = str(uuid.uuid4())
      crop_image_name = random_name + ".jpg"
      cv2.imwrite(output + "/" + random_name + ".jpg", cropped)
      f = open('/content/drive/MyDrive/recommendation image/rc_vintage_item.csv','a', newline='')
      wr = csv.writer(f)
      wr.writerow([crop_image_name, label, image_name])
      f.close

for num in range(num_images):
     print(f'{num+1} / {num_images}')
     try:
       image_name = file_list[num]
       frame = cv2.imread(path_dir+'/'+image_name, cv2.IMREAD_COLOR)
       frame_for_mask = cv2.imread(path_dir+'/'+image_name)
       results = model.detect([frame], verbose=0)      
       r = results[0]
       mask_crop(frame, frame_for_mask, r['masks'], r['class_ids'], class_names)
     except:
       except_file.append(file_list[num])

 print(f'except file: {except_file}')


